## House Scraping

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from datetime import datetime

def get_property_listings(url, page=1):
    full_url = f"{url}?page={page}"
    print(f"\nScraping URL: {full_url}")
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
    try:
        response = requests.get(full_url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        listings = soup.find_all('article', class_='listing-snippet')
        print(f"Found {len(listings)} listings on page {page}")
        return listings
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {str(e)}")
        return []

def extract_listing_data(listing):
    data = {}
    
    title_div = listing.find('div', class_='text-2xl font-semibold')
    data['title'] = title_div.get('title', '') if title_div else ''
    
    location_div = listing.find('div', class_='text-neutral-2 my-[5px]')
    data['location'] = location_div.text.strip() if location_div else ''
    
    price_div = listing.find('div', {'class': ['text-secondary-base', 'whitespace-nowrap', 'font-bold', 'text-3xl']})
    if price_div:
        data['price'] = price_div.text.strip()
        price_per_sqm_div = price_div.find_next_sibling('div', {'class': ['text-neutral-2', 'whitespace-nowrap', 'font-bold', 'text-xl']})
        if price_per_sqm_div:
            data['price_per_sqm'] = price_per_sqm_div.text.strip()
    else:
        data['price'] = ''
        data['price_per_sqm'] = ''
    
    link = listing.find('a', href=True)
    data['url'] = link['href'] if link else ''
    
    return data

def get_detailed_property_info(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        data = {}
        
        # Extract listing ID
        listing_id_p = soup.find('p', class_='internal-ref')
        if listing_id_p:
            listing_id = listing_id_p.text.strip().replace('Listing ID:', '').strip()
            data['listing_id'] = listing_id
        else:
            data['listing_id'] = ''
        
        key_features = soup.find('ul', class_='key-featured')
        if key_features:
            for item in key_features.find_all('li'):
                span_text = item.find_all('span')[-1].text.strip()
                if 'Bed:' in span_text or 'Beds:' in span_text:
                    data['bedrooms'] = span_text.replace('Bed:', '').replace('Beds:', '').strip()
                elif 'Bath:' in span_text or 'Baths:' in span_text:
                    data['bathrooms'] = span_text.replace('Bath:', '').replace('Baths:', '').strip()
                elif 'Usable area:' in span_text:
                    data['floor_area'] = span_text.replace('Usable area:', '').replace('m²', '').strip()
                elif 'Land area:' in span_text:
                    data['land_area'] = span_text.replace('Land area:', '').replace('m²', '').strip()
                elif 'Floor:' in span_text or 'Floors:' in span_text:
                    data['floor'] = span_text.replace('Floor:', '').replace('Floors:', '').strip()
        
        scripts = soup.find_all('script', type=lambda t: t and 'javascript' in t)
        for script in scripts:
            script_text = script.string if script.string else ''
            if 'var name =' in str(script_text):
                data['name'] = re.search('var name = "(.*?)";', str(script_text)).group(1)
            elif 'var gps_lat =' in str(script_text):
                data['latitude'] = re.search('var gps_lat = "(.*?)";', str(script_text)).group(1)
            elif 'var gps_lon =' in str(script_text):
                data['longitude'] = re.search('var gps_lon = "(.*?)";', str(script_text)).group(1)
            elif 'var address =' in str(script_text):
                data['address'] = re.search('var address = "(.*?)";', str(script_text)).group(1)
        
        facilities = []
        facility_items = soup.find_all('img', title=lambda t: t and 'Facilities:' in t)
        for item in facility_items:
            facility = item.get('title', '').replace('Facilities: ', '').strip()
            if facility:
                facilities.append(facility)
        
        data['facilities'] = facilities
        return data
        
    except Exception as e:
        print(f"Error fetching detailed info for {url}: {str(e)}")
        return {}

def create_facilities_columns(facilities_list):
    base_facilities = [
        'Access for the disabled', 'Air conditioning', 'Alarm', 'Balcony',
        'Built-in kitchen', 'Built-in wardrobe', 'Car park', "Children's area",
        'Cistern', 'Concierge', 'Electricity', 'Elevator', 'Equipped kitchen',
        'Garden', 'Grill', 'Guardhouse', 'Gym', 'Heating', 'Hot Tub', 'Internet',
        'Library', 'Natural gas', 'Office', 'Panoramic view', 'Patio',
        'Roof garden', 'Sauna', 'Security', 'Swimming pool', 'Tennis court',
        'Terrace', 'Utility room', 'Video cable', 'Water'
    ]
    
    facilities_dict = {facility: 0 for facility in base_facilities}
    for facility in facilities_list:
        if facility in facilities_dict:
            facilities_dict[facility] = 1
        else:
            print(f"Found new facility: {facility}")
            facilities_dict[facility] = 1
            
    return facilities_dict

def scrape_properties(base_url, max_pages=50):
    all_properties = []
    all_facilities = set()
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = f"dotproperty_house_listings_{timestamp}.csv"
    
    for page in range(1, max_pages + 1):
        listings = get_property_listings(base_url, page)
        if not listings:
            print(f"No listings found on page {page}")
            break
            
        for i, listing in enumerate(listings, 1):
            try:
                property_data = extract_listing_data(listing)
                if property_data['url']:
                    detailed_data = get_detailed_property_info(property_data['url'])
                    facilities = detailed_data.get('facilities', [])
                    all_facilities.update(facilities)
                    facilities_data = create_facilities_columns(facilities)
                    
                    property_data.update(detailed_data)
                    property_data.update(facilities_data)
                    all_properties.append(property_data)
                    
                    print(f"Property {i} on page {page} processed successfully")
                time.sleep(1)
            except Exception as e:
                print(f"Error processing property {i} on page {page}: {str(e)}")
        
        df = pd.DataFrame(all_properties)
        df.to_csv(output_file, index=False)
        print(f"Saved {len(all_properties)} listings to {output_file}")

    print(f"\nTotal unique facilities found: {len(all_facilities)}")
    print("Facility columns created:", sorted(list(all_facilities)))
    print(f"\nTotal listings scraped: {len(all_properties)}")
    
    return pd.DataFrame(all_properties)

if __name__ == "__main__":
    url = "https://www.dotproperty.com.ph/houses-for-sale/metro-manila/quezon-city"
    df = scrape_properties(url)
    print("Scraping completed!")


Scraping URL: https://www.dotproperty.com.ph/houses-for-sale/metro-manila/quezon-city?page=1
Found 25 listings on page 1
Property 1 on page 1 processed successfully
Property 2 on page 1 processed successfully
Property 3 on page 1 processed successfully
Property 4 on page 1 processed successfully
Property 5 on page 1 processed successfully
Property 6 on page 1 processed successfully
Property 7 on page 1 processed successfully
Property 8 on page 1 processed successfully
Property 9 on page 1 processed successfully
Property 10 on page 1 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 11 on page 1 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 12 on page 1 processed successfully
Property 13 on page 1 processed successfully
Property 14 on page 1 processed successfully
Property 15 on page 1 processed successfully
Found new facility: Cellar
Property 16 on page 1 processed successfully
Found new facility: Firepl

Property 7 on page 5 processed successfully
Property 8 on page 5 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 9 on page 5 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 10 on page 5 processed successfully
Property 11 on page 5 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 12 on page 5 processed successfully
Found new facility: Fireplace
Property 13 on page 5 processed successfully
Property 14 on page 5 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 15 on page 5 processed successfully
Property 16 on page 5 processed successfully
Property 17 on page 5 processed successfully
Property 18 on page 5 processed successfully
Property 19 on page 5 processed successfully
Found new facility: Cellar
Property 20 on page 5 processed successfully
Property 21 on page 5 processed successfully
Found new facility: Fireplace
Found new fac

Property 19 on page 10 processed successfully
Property 20 on page 10 processed successfully
Property 21 on page 10 processed successfully
Property 22 on page 10 processed successfully
Property 23 on page 10 processed successfully
Property 24 on page 10 processed successfully
Property 25 on page 10 processed successfully
Saved 250 listings to dotproperty_house_listings_20241201_023302.csv

Scraping URL: https://www.dotproperty.com.ph/houses-for-sale/metro-manila/quezon-city?page=11
Found 25 listings on page 11
Property 1 on page 11 processed successfully
Property 2 on page 11 processed successfully
Property 3 on page 11 processed successfully
Property 4 on page 11 processed successfully
Property 5 on page 11 processed successfully
Found new facility: Fireplace
Property 6 on page 11 processed successfully
Property 7 on page 11 processed successfully
Property 8 on page 11 processed successfully
Property 9 on page 11 processed successfully
Property 10 on page 11 processed successfully
Prop

Property 17 on page 16 processed successfully
Property 18 on page 16 processed successfully
Property 19 on page 16 processed successfully
Property 20 on page 16 processed successfully
Property 21 on page 16 processed successfully
Property 22 on page 16 processed successfully
Property 23 on page 16 processed successfully
Property 24 on page 16 processed successfully
Property 25 on page 16 processed successfully
Saved 400 listings to dotproperty_house_listings_20241201_023302.csv

Scraping URL: https://www.dotproperty.com.ph/houses-for-sale/metro-manila/quezon-city?page=17
Found 25 listings on page 17
Property 1 on page 17 processed successfully
Property 2 on page 17 processed successfully
Property 3 on page 17 processed successfully
Property 4 on page 17 processed successfully
Property 5 on page 17 processed successfully
Property 6 on page 17 processed successfully
Property 7 on page 17 processed successfully
Property 8 on page 17 processed successfully
Property 9 on page 17 processed s

Property 16 on page 22 processed successfully
Found new facility: Cellar
Property 17 on page 22 processed successfully
Found new facility: Cellar
Property 18 on page 22 processed successfully
Property 19 on page 22 processed successfully
Property 20 on page 22 processed successfully
Property 21 on page 22 processed successfully
Property 22 on page 22 processed successfully
Property 23 on page 22 processed successfully
Property 24 on page 22 processed successfully
Property 25 on page 22 processed successfully
Saved 550 listings to dotproperty_house_listings_20241201_023302.csv

Scraping URL: https://www.dotproperty.com.ph/houses-for-sale/metro-manila/quezon-city?page=23
Found 25 listings on page 23
Property 1 on page 23 processed successfully
Property 2 on page 23 processed successfully
Property 3 on page 23 processed successfully
Property 4 on page 23 processed successfully
Property 5 on page 23 processed successfully
Property 6 on page 23 processed successfully
Found new facility: Cel

Property 14 on page 28 processed successfully
Property 15 on page 28 processed successfully
Property 16 on page 28 processed successfully
Property 17 on page 28 processed successfully
Property 18 on page 28 processed successfully
Property 19 on page 28 processed successfully
Property 20 on page 28 processed successfully
Property 21 on page 28 processed successfully
Property 22 on page 28 processed successfully
Property 23 on page 28 processed successfully
Property 24 on page 28 processed successfully
Property 25 on page 28 processed successfully
Saved 700 listings to dotproperty_house_listings_20241201_023302.csv

Scraping URL: https://www.dotproperty.com.ph/houses-for-sale/metro-manila/quezon-city?page=29
Found 25 listings on page 29
Property 1 on page 29 processed successfully
Property 2 on page 29 processed successfully
Property 3 on page 29 processed successfully
Property 4 on page 29 processed successfully
Property 5 on page 29 processed successfully
Property 6 on page 29 processe

Property 17 on page 34 processed successfully
Property 18 on page 34 processed successfully
Property 19 on page 34 processed successfully
Property 20 on page 34 processed successfully
Property 21 on page 34 processed successfully
Property 22 on page 34 processed successfully
Property 23 on page 34 processed successfully
Property 24 on page 34 processed successfully
Property 25 on page 34 processed successfully
Saved 850 listings to dotproperty_house_listings_20241201_023302.csv

Scraping URL: https://www.dotproperty.com.ph/houses-for-sale/metro-manila/quezon-city?page=35
Found 25 listings on page 35
Property 1 on page 35 processed successfully
Property 2 on page 35 processed successfully
Property 3 on page 35 processed successfully
Property 4 on page 35 processed successfully
Property 5 on page 35 processed successfully
Property 6 on page 35 processed successfully
Property 7 on page 35 processed successfully
Property 8 on page 35 processed successfully
Property 9 on page 35 processed s

Property 20 on page 40 processed successfully
Property 21 on page 40 processed successfully
Property 22 on page 40 processed successfully
Property 23 on page 40 processed successfully
Property 24 on page 40 processed successfully
Property 25 on page 40 processed successfully
Saved 1000 listings to dotproperty_house_listings_20241201_023302.csv

Scraping URL: https://www.dotproperty.com.ph/houses-for-sale/metro-manila/quezon-city?page=41
Found 25 listings on page 41
Property 1 on page 41 processed successfully
Property 2 on page 41 processed successfully
Property 3 on page 41 processed successfully
Property 4 on page 41 processed successfully
Property 5 on page 41 processed successfully
Property 6 on page 41 processed successfully
Property 7 on page 41 processed successfully
Property 8 on page 41 processed successfully
Property 9 on page 41 processed successfully
Property 10 on page 41 processed successfully
Property 11 on page 41 processed successfully
Property 12 on page 41 processed 

Property 23 on page 46 processed successfully
Property 24 on page 46 processed successfully
Property 25 on page 46 processed successfully
Saved 1150 listings to dotproperty_house_listings_20241201_023302.csv

Scraping URL: https://www.dotproperty.com.ph/houses-for-sale/metro-manila/quezon-city?page=47
Found 25 listings on page 47
Property 1 on page 47 processed successfully
Property 2 on page 47 processed successfully
Property 3 on page 47 processed successfully
Property 4 on page 47 processed successfully
Property 5 on page 47 processed successfully
Property 6 on page 47 processed successfully
Property 7 on page 47 processed successfully
Property 8 on page 47 processed successfully
Property 9 on page 47 processed successfully
Property 10 on page 47 processed successfully
Property 11 on page 47 processed successfully
Property 12 on page 47 processed successfully
Property 13 on page 47 processed successfully
Property 14 on page 47 processed successfully
Property 15 on page 47 processed 

## Condo Scraping

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from datetime import datetime

def get_property_listings(url, page=1):
    # Handle the pagination parameter properly for the new URL
    if '?' in url:
        full_url = f"{url}&page={page}"
    else:
        full_url = f"{url}?page={page}"
    print(f"\nScraping URL: {full_url}")
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
    try:
        response = requests.get(full_url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        listings = soup.find_all('article', class_='listing-snippet')
        print(f"Found {len(listings)} listings on page {page}")
        return listings
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {str(e)}")
        return []

def extract_listing_data(listing):
    data = {}
    
    # Extract title
    title_div = listing.find('div', class_='text-2xl font-semibold')
    data['title'] = title_div.get('title', '') if title_div else ''
    
    # Extract location
    location_div = listing.find('div', class_='text-neutral-2 my-[5px]')
    data['location'] = location_div.text.strip() if location_div else ''
    
    # Extract price and price per sqm
    price_div = listing.find('div', {'class': ['text-secondary-base', 'whitespace-nowrap', 'font-bold', 'text-3xl']})
    if price_div:
        data['price'] = price_div.text.strip()
        price_per_sqm_div = price_div.find_next_sibling('div', {'class': ['text-neutral-2', 'whitespace-nowrap', 'font-bold', 'text-xl']})
        if price_per_sqm_div:
            data['price_per_sqm'] = price_per_sqm_div.text.strip()
    else:
        data['price'] = ''
        data['price_per_sqm'] = ''
    
    # Extract URL
    link = listing.find('a', href=True)
    data['url'] = link['href'] if link else ''
    
    return data

def get_detailed_property_info(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        data = {}
        
        # Extract listing ID
        listing_id_p = soup.find('p', class_='internal-ref')
        if listing_id_p:
            listing_id = listing_id_p.text.strip().replace('Listing ID:', '').strip()
            data['listing_id'] = listing_id
        else:
            data['listing_id'] = ''
        
        # Extract key features for condos
        key_features = soup.find('ul', class_='key-featured')
        if key_features:
            for item in key_features.find_all('li'):
                span_text = item.find_all('span')[-1].text.strip()
                if 'Bed:' in span_text or 'Beds:' in span_text:
                    data['bedrooms'] = span_text.replace('Bed:', '').replace('Beds:', '').strip()
                elif 'Bath:' in span_text or 'Baths:' in span_text:
                    data['bathrooms'] = span_text.replace('Bath:', '').replace('Baths:', '').strip()
                elif 'Usable area:' in span_text:
                    data['floor_area'] = span_text.replace('Usable area:', '').replace('m²', '').strip()
                elif 'Floor:' in span_text or 'Floors:' in span_text:
                    data['floor'] = span_text.replace('Floor:', '').replace('Floors:', '').strip()
        
        # Extract location data from scripts
        scripts = soup.find_all('script', type=lambda t: t and 'javascript' in t)
        for script in scripts:
            script_text = script.string if script.string else ''
            if 'var name =' in str(script_text):
                match = re.search('var name = "(.*?)";', str(script_text))
                if match:
                    data['name'] = match.group(1)
            elif 'var gps_lat =' in str(script_text):
                match = re.search('var gps_lat = "(.*?)";', str(script_text))
                if match:
                    data['latitude'] = match.group(1)
            elif 'var gps_lon =' in str(script_text):
                match = re.search('var gps_lon = "(.*?)";', str(script_text))
                if match:
                    data['longitude'] = match.group(1)
            elif 'var address =' in str(script_text):
                match = re.search('var address = "(.*?)";', str(script_text))
                if match:
                    data['address'] = match.group(1)
        
        # Extract facilities
        facilities = []
        facility_items = soup.find_all('img', title=lambda t: t and 'Facilities:' in t)
        for item in facility_items:
            facility = item.get('title', '').replace('Facilities: ', '').strip()
            if facility:
                facilities.append(facility)
        
        data['facilities'] = facilities
        return data
        
    except Exception as e:
        print(f"Error fetching detailed info for {url}: {str(e)}")
        return {}

def create_facilities_columns(facilities_list):
    # Updated base facilities list for condos based on the example
    base_facilities = [
        'Swimming pool', 'Air conditioning', 'Gym', 'Internet', 'Security',
        'Car park', 'Garden', 'Balcony', 'Tennis court', "Children's area",
        'Equipped kitchen', 'Electricity', 'Concierge', 'Elevator',
        'Roof garden', 'Water', 'Alarm', 'Access for the disabled',
        'Guardhouse', 'Built-in kitchen', 'Utility room', 'Video cable'
    ]
    
    facilities_dict = {facility: 0 for facility in base_facilities}
    for facility in facilities_list:
        if facility in facilities_dict:
            facilities_dict[facility] = 1
        else:
            print(f"Found new facility: {facility}")
            facilities_dict[facility] = 1
            
    return facilities_dict

def scrape_properties(base_url, max_pages=50):
    all_properties = []
    all_facilities = set()
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = f"dotproperty_condo_listings_{timestamp}.csv"
    
    for page in range(1, max_pages + 1):
        listings = get_property_listings(base_url, page)
        if not listings:
            print(f"No listings found on page {page}")
            break
            
        for i, listing in enumerate(listings, 1):
            try:
                property_data = extract_listing_data(listing)
                if property_data['url']:
                    detailed_data = get_detailed_property_info(property_data['url'])
                    facilities = detailed_data.get('facilities', [])
                    all_facilities.update(facilities)
                    facilities_data = create_facilities_columns(facilities)
                    
                    property_data.update(detailed_data)
                    property_data.update(facilities_data)
                    all_properties.append(property_data)
                    
                    print(f"Property {i} on page {page} processed successfully")
                time.sleep(1)
            except Exception as e:
                print(f"Error processing property {i} on page {page}: {str(e)}")
        
        df = pd.DataFrame(all_properties)
        df.to_csv(output_file, index=False)
        print(f"Saved {len(all_properties)} listings to {output_file}")

    print(f"\nTotal unique facilities found: {len(all_facilities)}")
    print("Facility columns created:", sorted(list(all_facilities)))
    print(f"\nTotal listings scraped: {len(all_properties)}")
    
    return pd.DataFrame(all_properties)

if __name__ == "__main__":
    url = "https://www.dotproperty.com.ph/condos-for-sale/metro-manila/quezon-city?exact_bed=false"
    df = scrape_properties(url)
    print("Scraping completed!")


Scraping URL: https://www.dotproperty.com.ph/condos-for-sale/metro-manila/quezon-city?exact_bed=false&page=1
Found 25 listings on page 1
Found new facility: Patio
Found new facility: Cistern
Property 1 on page 1 processed successfully
Property 2 on page 1 processed successfully
Found new facility: Panoramic view
Found new facility: Library
Found new facility: Terrace
Found new facility: Patio
Found new facility: Heating
Found new facility: Fireplace
Found new facility: Built-in wardrobe
Found new facility: Cellar
Found new facility: Grill
Found new facility: Hot Tub
Found new facility: Natural gas
Found new facility: Office
Found new facility: Cistern
Property 3 on page 1 processed successfully
Found new facility: Sauna
Found new facility: Panoramic view
Found new facility: Library
Found new facility: Terrace
Found new facility: Patio
Found new facility: Heating
Found new facility: Fireplace
Found new facility: Built-in wardrobe
Found new facility: Cellar
Found new facility: Grill
Fou

Property 13 on page 11 processed successfully
Found new facility: Fireplace
Property 14 on page 11 processed successfully
Property 15 on page 11 processed successfully
Property 16 on page 11 processed successfully
Property 17 on page 11 processed successfully
Property 18 on page 11 processed successfully
Property 19 on page 11 processed successfully
Property 20 on page 11 processed successfully
Property 21 on page 11 processed successfully
Property 22 on page 11 processed successfully
Property 23 on page 11 processed successfully
Property 24 on page 11 processed successfully
Property 25 on page 11 processed successfully
Saved 275 listings to dotproperty_condo_listings_20241201_033433.csv

Scraping URL: https://www.dotproperty.com.ph/condos-for-sale/metro-manila/quezon-city?exact_bed=false?page=12
Found 25 listings on page 12
Property 1 on page 12 processed successfully
Property 2 on page 12 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 3 on pa

Property 19 on page 16 processed successfully
Property 20 on page 16 processed successfully
Property 21 on page 16 processed successfully
Property 22 on page 16 processed successfully
Property 23 on page 16 processed successfully
Property 24 on page 16 processed successfully
Property 25 on page 16 processed successfully
Saved 400 listings to dotproperty_condo_listings_20241201_033433.csv

Scraping URL: https://www.dotproperty.com.ph/condos-for-sale/metro-manila/quezon-city?exact_bed=false?page=17
Found 25 listings on page 17
Property 1 on page 17 processed successfully
Property 2 on page 17 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 3 on page 17 processed successfully
Property 4 on page 17 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 5 on page 17 processed successfully
Property 6 on page 17 processed successfully
Property 7 on page 17 processed successfully
Property 8 on page 17 processed successf

Saved 525 listings to dotproperty_condo_listings_20241201_033433.csv

Scraping URL: https://www.dotproperty.com.ph/condos-for-sale/metro-manila/quezon-city?exact_bed=false?page=22
Found 25 listings on page 22
Property 1 on page 22 processed successfully
Property 2 on page 22 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 3 on page 22 processed successfully
Property 4 on page 22 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 5 on page 22 processed successfully
Property 6 on page 22 processed successfully
Property 7 on page 22 processed successfully
Property 8 on page 22 processed successfully
Property 9 on page 22 processed successfully
Property 10 on page 22 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 11 on page 22 processed successfully
Found new facility: Fireplace
Property 12 on page 22 processed successfully
Property 13 on page 22 processed successfully
F

Found new facility: Fireplace
Found new facility: Cellar
Property 3 on page 27 processed successfully
Property 4 on page 27 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 5 on page 27 processed successfully
Property 6 on page 27 processed successfully
Property 7 on page 27 processed successfully
Property 8 on page 27 processed successfully
Property 9 on page 27 processed successfully
Property 10 on page 27 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 11 on page 27 processed successfully
Found new facility: Fireplace
Property 12 on page 27 processed successfully
Property 13 on page 27 processed successfully
Found new facility: Fireplace
Property 14 on page 27 processed successfully
Property 15 on page 27 processed successfully
Property 16 on page 27 processed successfully
Property 17 on page 27 processed successfully
Property 18 on page 27 processed successfully
Property 19 on page 27 processed success

Property 7 on page 32 processed successfully
Property 8 on page 32 processed successfully
Property 9 on page 32 processed successfully
Property 10 on page 32 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 11 on page 32 processed successfully
Found new facility: Fireplace
Property 12 on page 32 processed successfully
Property 13 on page 32 processed successfully
Found new facility: Fireplace
Property 14 on page 32 processed successfully
Property 15 on page 32 processed successfully
Property 16 on page 32 processed successfully
Property 17 on page 32 processed successfully
Property 18 on page 32 processed successfully
Property 19 on page 32 processed successfully
Property 20 on page 32 processed successfully
Property 21 on page 32 processed successfully
Property 22 on page 32 processed successfully
Property 23 on page 32 processed successfully
Property 24 on page 32 processed successfully
Property 25 on page 32 processed successfully
Saved 800 li

Property 13 on page 37 processed successfully
Found new facility: Fireplace
Property 14 on page 37 processed successfully
Property 15 on page 37 processed successfully
Property 16 on page 37 processed successfully
Property 17 on page 37 processed successfully
Property 18 on page 37 processed successfully
Property 19 on page 37 processed successfully
Property 20 on page 37 processed successfully
Property 21 on page 37 processed successfully
Property 22 on page 37 processed successfully
Property 23 on page 37 processed successfully
Property 24 on page 37 processed successfully
Property 25 on page 37 processed successfully
Saved 925 listings to dotproperty_condo_listings_20241201_033433.csv

Scraping URL: https://www.dotproperty.com.ph/condos-for-sale/metro-manila/quezon-city?exact_bed=false?page=38
Found 25 listings on page 38
Property 1 on page 38 processed successfully
Property 2 on page 38 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 3 on pa

Property 20 on page 42 processed successfully
Property 21 on page 42 processed successfully
Property 22 on page 42 processed successfully
Property 23 on page 42 processed successfully
Property 24 on page 42 processed successfully
Property 25 on page 42 processed successfully
Saved 1050 listings to dotproperty_condo_listings_20241201_033433.csv

Scraping URL: https://www.dotproperty.com.ph/condos-for-sale/metro-manila/quezon-city?exact_bed=false?page=43
Found 25 listings on page 43
Property 1 on page 43 processed successfully
Property 2 on page 43 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 3 on page 43 processed successfully
Property 4 on page 43 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 5 on page 43 processed successfully
Property 6 on page 43 processed successfully
Property 7 on page 43 processed successfully
Property 8 on page 43 processed successfully
Property 9 on page 43 processed successf

Found 25 listings on page 48
Property 1 on page 48 processed successfully
Property 2 on page 48 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 3 on page 48 processed successfully
Property 4 on page 48 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 5 on page 48 processed successfully
Property 6 on page 48 processed successfully
Property 7 on page 48 processed successfully
Property 8 on page 48 processed successfully
Property 9 on page 48 processed successfully
Property 10 on page 48 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 11 on page 48 processed successfully
Found new facility: Fireplace
Property 12 on page 48 processed successfully
Property 13 on page 48 processed successfully
Property 14 on page 48 processed successfully
Property 15 on page 48 processed successfully
Property 16 on page 48 processed successfully
Property 17 on page 48 processed successful

## Apartment Scraping

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from datetime import datetime

def get_property_listings(url, page=1):
    # Handle the pagination parameter properly for the new URL
    if '?' in url:
        full_url = f"{url}&page={page}"
    else:
        full_url = f"{url}?page={page}"
    print(f"\nScraping URL: {full_url}")
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
    try:
        response = requests.get(full_url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        listings = soup.find_all('article', class_='listing-snippet')
        print(f"Found {len(listings)} listings on page {page}")
        return listings
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {str(e)}")
        return []

def extract_listing_data(listing):
    data = {}
    
    # Extract title
    title_div = listing.find('div', class_='text-2xl font-semibold')
    data['title'] = title_div.get('title', '') if title_div else ''
    
    # Extract location
    location_div = listing.find('div', class_='text-neutral-2 my-[5px]')
    data['location'] = location_div.text.strip() if location_div else ''
    
    # Extract price and price per sqm
    price_div = listing.find('div', {'class': ['text-secondary-base', 'whitespace-nowrap', 'font-bold', 'text-3xl']})
    if price_div:
        data['price'] = price_div.text.strip()
        price_per_sqm_div = price_div.find_next_sibling('div', {'class': ['text-neutral-2', 'whitespace-nowrap', 'font-bold', 'text-xl']})
        if price_per_sqm_div:
            data['price_per_sqm'] = price_per_sqm_div.text.strip()
    else:
        data['price'] = ''
        data['price_per_sqm'] = ''
    
    # Extract URL
    link = listing.find('a', href=True)
    data['url'] = link['href'] if link else ''
    
    return data

def get_detailed_property_info(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        data = {}
        
        # Extract listing ID
        listing_id_p = soup.find('p', class_='internal-ref')
        if listing_id_p:
            listing_id = listing_id_p.text.strip().replace('Listing ID:', '').strip()
            data['listing_id'] = listing_id
        else:
            data['listing_id'] = ''
        
        # Extract key features
        key_features = soup.find('ul', class_='key-featured')
        if key_features:
            for item in key_features.find_all('li'):
                span_text = item.find_all('span')[-1].text.strip()
                if 'Bed:' in span_text or 'Beds:' in span_text:
                    data['bedrooms'] = span_text.replace('Bed:', '').replace('Beds:', '').strip()
                elif 'Bath:' in span_text or 'Baths:' in span_text:
                    data['bathrooms'] = span_text.replace('Bath:', '').replace('Baths:', '').strip()
                elif 'Usable area:' in span_text:
                    data['floor_area'] = span_text.replace('Usable area:', '').replace('m²', '').strip()
                elif 'Floor:' in span_text or 'Floors:' in span_text:
                    data['floor'] = span_text.replace('Floor:', '').replace('Floors:', '').strip()
        
        # Extract location data from scripts
        scripts = soup.find_all('script', type=lambda t: t and 'javascript' in t)
        for script in scripts:
            script_text = script.string if script.string else ''
            if 'var name =' in str(script_text):
                match = re.search('var name = "(.*?)";', str(script_text))
                if match:
                    data['name'] = match.group(1)
            elif 'var gps_lat =' in str(script_text):
                match = re.search('var gps_lat = "(.*?)";', str(script_text))
                if match:
                    data['latitude'] = match.group(1)
            elif 'var gps_lon =' in str(script_text):
                match = re.search('var gps_lon = "(.*?)";', str(script_text))
                if match:
                    data['longitude'] = match.group(1)
            elif 'var address =' in str(script_text):
                match = re.search('var address = "(.*?)";', str(script_text))
                if match:
                    data['address'] = match.group(1)
        
        # Extract facilities
        facilities = []
        facility_items = soup.find_all('img', title=lambda t: t and 'Facilities:' in t)
        for item in facility_items:
            facility = item.get('title', '').replace('Facilities: ', '').strip()
            if facility:
                facilities.append(facility)
        
        data['facilities'] = facilities
        return data
        
    except Exception as e:
        print(f"Error fetching detailed info for {url}: {str(e)}")
        return {}

def create_facilities_columns(facilities_list):
    # Updated base facilities list based on the example apartment listing
    base_facilities = [
        'Internet', 'Car park', 'Balcony', 'Electricity', 'Panoramic view',
        'Terrace', 'Water', 'Cistern', 'Air conditioning', 'Security',
        'Built-in kitchen', 'Equipped kitchen', 'Utility room', 'Video cable',
        'Access for the disabled', 'Alarm', 'Built-in wardrobe', 'Elevator'
    ]
    
    facilities_dict = {facility: 0 for facility in base_facilities}
    for facility in facilities_list:
        if facility in facilities_dict:
            facilities_dict[facility] = 1
        else:
            print(f"Found new facility: {facility}")
            facilities_dict[facility] = 1
            
    return facilities_dict

def scrape_properties(base_url, max_pages=10):
    all_properties = []
    all_facilities = set()
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = f"dotproperty_apartment_listings_{timestamp}.csv"
    
    for page in range(1, max_pages + 1):
        listings = get_property_listings(base_url, page)
        if not listings:
            print(f"No listings found on page {page}")
            break
            
        for i, listing in enumerate(listings, 1):
            try:
                property_data = extract_listing_data(listing)
                if property_data['url']:
                    detailed_data = get_detailed_property_info(property_data['url'])
                    facilities = detailed_data.get('facilities', [])
                    all_facilities.update(facilities)
                    facilities_data = create_facilities_columns(facilities)
                    
                    property_data.update(detailed_data)
                    property_data.update(facilities_data)
                    all_properties.append(property_data)
                    
                    print(f"Property {i} on page {page} processed successfully")
                time.sleep(1)
            except Exception as e:
                print(f"Error processing property {i} on page {page}: {str(e)}")
        
        df = pd.DataFrame(all_properties)
        df.to_csv(output_file, index=False)
        print(f"Saved {len(all_properties)} listings to {output_file}")

    print(f"\nTotal unique facilities found: {len(all_facilities)}")
    print("Facility columns created:", sorted(list(all_facilities)))
    print(f"\nTotal listings scraped: {len(all_properties)}")
    
    return pd.DataFrame(all_properties)

if __name__ == "__main__":
    url = "https://www.dotproperty.com.ph/apartments-for-sale/metro-manila/quezon-city?exact_bed=false"
    df = scrape_properties(url)
    print("Scraping completed!")


Scraping URL: https://www.dotproperty.com.ph/apartments-for-sale/metro-manila/quezon-city?exact_bed=false&page=1
Found 25 listings on page 1
Property 1 on page 1 processed successfully
Found new facility: Swimming pool
Found new facility: Gym
Found new facility: Garden
Found new facility: Children's area
Found new facility: Concierge
Found new facility: Library
Property 2 on page 1 processed successfully
Property 3 on page 1 processed successfully
Property 4 on page 1 processed successfully
Property 5 on page 1 processed successfully
Property 6 on page 1 processed successfully
Property 7 on page 1 processed successfully
Property 8 on page 1 processed successfully
Found new facility: Swimming pool
Found new facility: Gym
Found new facility: Garden
Found new facility: Tennis court
Found new facility: Sauna
Found new facility: Children's area
Found new facility: Concierge
Found new facility: Library
Found new facility: Roof garden
Found new facility: Patio
Found new facility: Heating
Fou

Found new facility: Fireplace
Found new facility: Cellar
Property 11 on page 10 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 12 on page 10 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 13 on page 10 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 14 on page 10 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 15 on page 10 processed successfully
Property 16 on page 10 processed successfully
Found new facility: Cellar
Property 17 on page 10 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 18 on page 10 processed successfully
Property 19 on page 10 processed successfully
Property 20 on page 10 processed successfully
Property 21 on page 10 processed successfully
Property 22 on page 10 processed successfully
Property 23 on page 10 processed successfully
Property 24 on page 10 processed 

Found 25 listings on page 15
Property 1 on page 15 processed successfully
Property 2 on page 15 processed successfully
Property 3 on page 15 processed successfully
Property 4 on page 15 processed successfully
Property 5 on page 15 processed successfully
Property 6 on page 15 processed successfully
Property 7 on page 15 processed successfully
Property 8 on page 15 processed successfully
Property 9 on page 15 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 10 on page 15 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 11 on page 15 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 12 on page 15 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 13 on page 15 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 14 on page 15 processed successfully
Found new facility: Fireplace
Found new facility: C

Found new facility: Cellar
Property 17 on page 19 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 18 on page 19 processed successfully
Property 19 on page 19 processed successfully
Property 20 on page 19 processed successfully
Property 21 on page 19 processed successfully
Property 22 on page 19 processed successfully
Property 23 on page 19 processed successfully
Property 24 on page 19 processed successfully
Property 25 on page 19 processed successfully
Saved 475 listings to dotproperty_apt_listings_20241201_043652.csv

Scraping URL: https://www.dotproperty.com.ph/apartments-for-sale/metro-manila/quezon-city?exact_bed=false?page=20
Found 25 listings on page 20
Property 1 on page 20 processed successfully
Property 2 on page 20 processed successfully
Property 3 on page 20 processed successfully
Property 4 on page 20 processed successfully
Property 5 on page 20 processed successfully
Property 6 on page 20 processed successfully
Property 7 on page 20

Found new facility: Fireplace
Found new facility: Cellar
Property 10 on page 24 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 11 on page 24 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 12 on page 24 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 13 on page 24 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 14 on page 24 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 15 on page 24 processed successfully
Property 16 on page 24 processed successfully
Found new facility: Cellar
Property 17 on page 24 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 18 on page 24 processed successfully
Property 19 on page 24 processed successfully
Property 20 on page 24 processed successfully
Property 21 on page 24 processed successfully
Property 22 on page 24

Property 25 on page 28 processed successfully
Saved 700 listings to dotproperty_apt_listings_20241201_043652.csv

Scraping URL: https://www.dotproperty.com.ph/apartments-for-sale/metro-manila/quezon-city?exact_bed=false?page=29
Found 25 listings on page 29
Property 1 on page 29 processed successfully
Property 2 on page 29 processed successfully
Property 3 on page 29 processed successfully
Property 4 on page 29 processed successfully
Property 5 on page 29 processed successfully
Property 6 on page 29 processed successfully
Property 7 on page 29 processed successfully
Property 8 on page 29 processed successfully
Property 9 on page 29 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 10 on page 29 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 11 on page 29 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 12 on page 29 processed successfully
Found new facility: Fireplace

Found new facility: Fireplace
Found new facility: Cellar
Property 15 on page 33 processed successfully
Property 16 on page 33 processed successfully
Found new facility: Cellar
Property 17 on page 33 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 18 on page 33 processed successfully
Property 19 on page 33 processed successfully
Property 20 on page 33 processed successfully
Property 21 on page 33 processed successfully
Property 22 on page 33 processed successfully
Property 23 on page 33 processed successfully
Property 24 on page 33 processed successfully
Property 25 on page 33 processed successfully
Saved 825 listings to dotproperty_apt_listings_20241201_043652.csv

Scraping URL: https://www.dotproperty.com.ph/apartments-for-sale/metro-manila/quezon-city?exact_bed=false?page=34
Found 25 listings on page 34
Property 1 on page 34 processed successfully
Property 2 on page 34 processed successfully
Property 3 on page 34 processed successfully
Propert

Property 6 on page 38 processed successfully
Property 7 on page 38 processed successfully
Property 8 on page 38 processed successfully
Property 9 on page 38 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 10 on page 38 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 11 on page 38 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 12 on page 38 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 13 on page 38 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 14 on page 38 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 15 on page 38 processed successfully
Property 16 on page 38 processed successfully
Found new facility: Cellar
Property 17 on page 38 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 18 on page 38 pro

Property 21 on page 42 processed successfully
Property 22 on page 42 processed successfully
Property 23 on page 42 processed successfully
Property 24 on page 42 processed successfully
Property 25 on page 42 processed successfully
Saved 1050 listings to dotproperty_apt_listings_20241201_043652.csv

Scraping URL: https://www.dotproperty.com.ph/apartments-for-sale/metro-manila/quezon-city?exact_bed=false?page=43
Found 25 listings on page 43
Property 1 on page 43 processed successfully
Property 2 on page 43 processed successfully
Property 3 on page 43 processed successfully
Property 4 on page 43 processed successfully
Property 5 on page 43 processed successfully
Property 6 on page 43 processed successfully
Property 7 on page 43 processed successfully
Property 8 on page 43 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 9 on page 43 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 10 on page 43 processed succes

Found new facility: Fireplace
Found new facility: Cellar
Property 12 on page 47 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 13 on page 47 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 14 on page 47 processed successfully
Property 15 on page 47 processed successfully
Property 16 on page 47 processed successfully
Found new facility: Fireplace
Found new facility: Cellar
Property 17 on page 47 processed successfully
Found new facility: Cellar
Property 18 on page 47 processed successfully
Property 19 on page 47 processed successfully
Property 20 on page 47 processed successfully
Property 21 on page 47 processed successfully
Property 22 on page 47 processed successfully
Property 23 on page 47 processed successfully
Property 24 on page 47 processed successfully
Property 25 on page 47 processed successfully
Saved 1175 listings to dotproperty_apt_listings_20241201_043652.csv

Scraping URL: https://www.dotpro

## Commercial Scraping

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from datetime import datetime

def get_property_listings(url, page=1):
    # Handle the pagination parameter properly for the new URL
    if '?' in url:
        full_url = f"{url}&page={page}"
    else:
        full_url = f"{url}?page={page}"
    print(f"\nScraping URL: {full_url}")
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
    try:
        response = requests.get(full_url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        listings = soup.find_all('article', class_='listing-snippet')
        print(f"Found {len(listings)} listings on page {page}")
        return listings
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {str(e)}")
        return []

def extract_listing_data(listing):
    data = {}
    
    # Extract title
    title_div = listing.find('div', class_='text-2xl font-semibold')
    data['title'] = title_div.get('title', '') if title_div else ''
    
    # Extract location
    location_div = listing.find('div', class_='text-neutral-2 my-[5px]')
    data['location'] = location_div.text.strip() if location_div else ''
    
    # Extract price and price per sqm
    price_div = listing.find('div', {'class': ['text-secondary-base', 'whitespace-nowrap', 'font-bold', 'text-3xl']})
    if price_div:
        data['price'] = price_div.text.strip()
        price_per_sqm_div = price_div.find_next_sibling('div', {'class': ['text-neutral-2', 'whitespace-nowrap', 'font-bold', 'text-xl']})
        if price_per_sqm_div:
            data['price_per_sqm'] = price_per_sqm_div.text.strip()
    else:
        data['price'] = ''
        data['price_per_sqm'] = ''
    
    # Extract URL
    link = listing.find('a', href=True)
    data['url'] = link['href'] if link else ''
    
    return data

def get_detailed_property_info(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        data = {}
        
        # Extract listing ID
        listing_id_p = soup.find('p', class_='internal-ref')
        if listing_id_p:
            listing_id = listing_id_p.text.strip().replace('Listing ID:', '').strip()
            data['listing_id'] = listing_id
        else:
            data['listing_id'] = ''
        
        # Extract key features
        key_features = soup.find('ul', class_='key-featured')
        if key_features:
            for item in key_features.find_all('li'):
                span_text = item.find_all('span')[-1].text.strip()
                if 'Bed:' in span_text or 'Beds:' in span_text:
                    data['bedrooms'] = span_text.replace('Bed:', '').replace('Beds:', '').strip()
                elif 'Bath:' in span_text or 'Baths:' in span_text:
                    data['bathrooms'] = span_text.replace('Bath:', '').replace('Baths:', '').strip()
                elif 'Usable area:' in span_text:
                    data['floor_area'] = span_text.replace('Usable area:', '').replace('m²', '').strip()
                elif 'Land area:' in span_text:
                    data['land_area'] = span_text.replace('Land area:', '').replace('m²', '').strip()
                elif 'Floor:' in span_text or 'Floors:' in span_text:
                    data['floor'] = span_text.replace('Floor:', '').replace('Floors:', '').strip()
        
        # Extract location data from scripts
        scripts = soup.find_all('script', type=lambda t: t and 'javascript' in t)
        for script in scripts:
            script_text = script.string if script.string else ''
            if 'var name =' in str(script_text):
                match = re.search('var name = "(.*?)";', str(script_text))
                if match:
                    data['name'] = match.group(1)
            elif 'var gps_lat =' in str(script_text):
                match = re.search('var gps_lat = "(.*?)";', str(script_text))
                if match:
                    data['latitude'] = match.group(1)
            elif 'var gps_lon =' in str(script_text):
                match = re.search('var gps_lon = "(.*?)";', str(script_text))
                if match:
                    data['longitude'] = match.group(1)
            elif 'var address =' in str(script_text):
                match = re.search('var address = "(.*?)";', str(script_text))
                if match:
                    data['address'] = match.group(1)
        
        # Extract facilities
        facilities = []
        facility_items = soup.find_all('img', title=lambda t: t and 'Facilities:' in t)
        for item in facility_items:
            facility = item.get('title', '').replace('Facilities: ', '').strip()
            if facility:
                facilities.append(facility)
        
        data['facilities'] = facilities
        return data
        
    except Exception as e:
        print(f"Error fetching detailed info for {url}: {str(e)}")
        return {}

def create_facilities_columns(facilities_list):
    # Updated base facilities list for commercial properties
    base_facilities = [
        'Access for the disabled', 'Air conditioning', 'Alarm', 'Car park',
        'Concierge', 'Electricity', 'Elevator', 'Internet', 'Office',
        'Security', 'Water', 'Built-in kitchen', 'Equipped kitchen',
        'Garden', 'Guardhouse', 'Gym', 'Swimming pool'
    ]
    
    facilities_dict = {facility: 0 for facility in base_facilities}
    for facility in facilities_list:
        if facility in facilities_dict:
            facilities_dict[facility] = 1
        else:
            print(f"Found new facility: {facility}")
            facilities_dict[facility] = 1
            
    return facilities_dict

def scrape_properties(base_url, max_pages=50):
    all_properties = []
    all_facilities = set()
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = f"dotproperty_commercial_listings_{timestamp}.csv"
    
    for page in range(1, max_pages + 1):
        listings = get_property_listings(base_url, page)
        if not listings:
            print(f"No listings found on page {page}")
            break
            
        for i, listing in enumerate(listings, 1):
            try:
                property_data = extract_listing_data(listing)
                if property_data['url']:
                    detailed_data = get_detailed_property_info(property_data['url'])
                    facilities = detailed_data.get('facilities', [])
                    all_facilities.update(facilities)
                    facilities_data = create_facilities_columns(facilities)
                    
                    property_data.update(detailed_data)
                    property_data.update(facilities_data)
                    all_properties.append(property_data)
                    
                    print(f"Property {i} on page {page} processed successfully")
                time.sleep(1)
            except Exception as e:
                print(f"Error processing property {i} on page {page}: {str(e)}")
        
        df = pd.DataFrame(all_properties)
        df.to_csv(output_file, index=False)
        print(f"Saved {len(all_properties)} listings to {output_file}")

    print(f"\nTotal unique facilities found: {len(all_facilities)}")
    print("Facility columns created:", sorted(list(all_facilities)))
    print(f"\nTotal listings scraped: {len(all_properties)}")
    
    return pd.DataFrame(all_properties)

if __name__ == "__main__":
    url = "https://www.dotproperty.com.ph/commercial-property-for-sale/metro-manila/quezon-city?exact_bed=false"
    df = scrape_properties(url)
    print("Scraping completed!")


Scraping URL: https://www.dotproperty.com.ph/commercial-property-for-sale/metro-manila/quezon-city?exact_bed=false&page=1
Found 25 listings on page 1
Property 1 on page 1 processed successfully
Property 2 on page 1 processed successfully
Property 3 on page 1 processed successfully
Property 4 on page 1 processed successfully
Found new facility: Tennis court
Found new facility: Children's area
Found new facility: Library
Found new facility: Grill
Found new facility: Cistern
Property 5 on page 1 processed successfully
Property 6 on page 1 processed successfully
Property 7 on page 1 processed successfully
Found new facility: Grill
Found new facility: Cistern
Property 8 on page 1 processed successfully
Property 9 on page 1 processed successfully
Property 10 on page 1 processed successfully
Found new facility: Tennis court
Found new facility: Children's area
Found new facility: Cistern
Property 11 on page 1 processed successfully
Found new facility: Children's area
Found new facility: Ciste

Property 51 on page 13 processed successfully
Property 52 on page 13 processed successfully
Property 53 on page 13 processed successfully
Property 54 on page 13 processed successfully
Property 55 on page 13 processed successfully
Property 56 on page 13 processed successfully
Property 57 on page 13 processed successfully
Property 58 on page 13 processed successfully
Property 59 on page 13 processed successfully
Property 60 on page 13 processed successfully
Property 61 on page 13 processed successfully
Property 62 on page 13 processed successfully
Property 63 on page 13 processed successfully
Property 64 on page 13 processed successfully
Property 65 on page 13 processed successfully
Property 66 on page 13 processed successfully
Property 67 on page 13 processed successfully
Property 68 on page 13 processed successfully
Property 69 on page 13 processed successfully
Property 70 on page 13 processed successfully
Property 71 on page 13 processed successfully
Property 72 on page 13 processed s

Property 51 on page 19 processed successfully
Property 52 on page 19 processed successfully
Property 53 on page 19 processed successfully
Property 54 on page 19 processed successfully
Property 55 on page 19 processed successfully
Property 56 on page 19 processed successfully
Property 57 on page 19 processed successfully
Property 58 on page 19 processed successfully
Property 59 on page 19 processed successfully
Property 60 on page 19 processed successfully
Property 61 on page 19 processed successfully
Property 62 on page 19 processed successfully
Property 63 on page 19 processed successfully
Property 64 on page 19 processed successfully
Property 65 on page 19 processed successfully
Property 66 on page 19 processed successfully
Property 67 on page 19 processed successfully
Property 68 on page 19 processed successfully
Property 69 on page 19 processed successfully
Property 70 on page 19 processed successfully
Property 71 on page 19 processed successfully
Property 72 on page 19 processed s

Property 51 on page 25 processed successfully
Property 52 on page 25 processed successfully
Property 53 on page 25 processed successfully
Property 54 on page 25 processed successfully
Property 55 on page 25 processed successfully
Property 56 on page 25 processed successfully
Property 57 on page 25 processed successfully
Property 58 on page 25 processed successfully
Property 59 on page 25 processed successfully
Property 60 on page 25 processed successfully
Property 61 on page 25 processed successfully
Property 62 on page 25 processed successfully
Property 63 on page 25 processed successfully
Property 64 on page 25 processed successfully
Property 65 on page 25 processed successfully
Property 66 on page 25 processed successfully
Property 67 on page 25 processed successfully
Property 68 on page 25 processed successfully
Property 69 on page 25 processed successfully
Property 70 on page 25 processed successfully
Property 71 on page 25 processed successfully
Property 72 on page 25 processed s

Property 51 on page 31 processed successfully
Property 52 on page 31 processed successfully
Property 53 on page 31 processed successfully
Property 54 on page 31 processed successfully
Property 55 on page 31 processed successfully
Property 56 on page 31 processed successfully
Property 57 on page 31 processed successfully
Property 58 on page 31 processed successfully
Property 59 on page 31 processed successfully
Property 60 on page 31 processed successfully
Property 61 on page 31 processed successfully
Property 62 on page 31 processed successfully
Property 63 on page 31 processed successfully
Property 64 on page 31 processed successfully
Property 65 on page 31 processed successfully
Property 66 on page 31 processed successfully
Property 67 on page 31 processed successfully
Property 68 on page 31 processed successfully
Property 69 on page 31 processed successfully
Property 70 on page 31 processed successfully
Property 71 on page 31 processed successfully
Property 72 on page 31 processed s

Property 51 on page 37 processed successfully
Property 52 on page 37 processed successfully
Property 53 on page 37 processed successfully
Property 54 on page 37 processed successfully
Property 55 on page 37 processed successfully
Property 56 on page 37 processed successfully
Property 57 on page 37 processed successfully
Property 58 on page 37 processed successfully
Property 59 on page 37 processed successfully
Property 60 on page 37 processed successfully
Property 61 on page 37 processed successfully
Property 62 on page 37 processed successfully
Property 63 on page 37 processed successfully
Property 64 on page 37 processed successfully
Property 65 on page 37 processed successfully
Property 66 on page 37 processed successfully
Property 67 on page 37 processed successfully
Property 68 on page 37 processed successfully
Property 69 on page 37 processed successfully
Property 70 on page 37 processed successfully
Property 71 on page 37 processed successfully
Property 72 on page 37 processed s

Property 51 on page 43 processed successfully
Property 52 on page 43 processed successfully
Property 53 on page 43 processed successfully
Property 54 on page 43 processed successfully
Property 55 on page 43 processed successfully
Property 56 on page 43 processed successfully
Property 57 on page 43 processed successfully
Property 58 on page 43 processed successfully
Property 59 on page 43 processed successfully
Property 60 on page 43 processed successfully
Property 61 on page 43 processed successfully
Property 62 on page 43 processed successfully
Property 63 on page 43 processed successfully
Property 64 on page 43 processed successfully
Property 65 on page 43 processed successfully
Property 66 on page 43 processed successfully
Property 67 on page 43 processed successfully
Property 68 on page 43 processed successfully
Property 69 on page 43 processed successfully
Property 70 on page 43 processed successfully
Property 71 on page 43 processed successfully
Property 72 on page 43 processed s

Property 51 on page 49 processed successfully
Property 52 on page 49 processed successfully
Property 53 on page 49 processed successfully
Property 54 on page 49 processed successfully
Property 55 on page 49 processed successfully
Property 56 on page 49 processed successfully
Property 57 on page 49 processed successfully
Property 58 on page 49 processed successfully
Property 59 on page 49 processed successfully
Property 60 on page 49 processed successfully
Property 61 on page 49 processed successfully
Property 62 on page 49 processed successfully
Property 63 on page 49 processed successfully
Property 64 on page 49 processed successfully
Property 65 on page 49 processed successfully
Property 66 on page 49 processed successfully
Property 67 on page 49 processed successfully
Property 68 on page 49 processed successfully
Property 69 on page 49 processed successfully
Property 70 on page 49 processed successfully
Property 71 on page 49 processed successfully
Property 72 on page 49 processed s